In [ ]:
import gmaps
import gmaps.datasets
import gmaps.geojson_geometries
import json
from matplotlib.cm import viridis, plasma
from matplotlib.colors import to_hex
import matplotlib.pyplot as plt
import pandas as pd
from config import census_key, g_key
pd.set_option('display.max_columns', None)
gmaps.configure(api_key=g_key)

## Create State Info Table

In [ ]:


# Create file with state info including codes and names

states_json = pd.read_json('data/us_states.json')

objects = states_json.features.to_list()

state_codes = []
state_names = []
state_census_area = []

for object in objects:
    p = object['properties']
    state_codes.append(p['STATE'])
    state_names.append(p['NAME'])
    state_census_area.append(p['CENSUSAREA'])

state_codes = pd.DataFrame(
    {'State Codes': state_codes,
    'State Names': state_names,
    'State Census Area': state_census_area
    })

# Import state abbrievations

state_abbreviations = pd.read_csv('data/state_abbreviations.csv')

state_abbreviations[state_abbreviations =='D.C.'] = 'District of Columbia'

state_abbreviations.rename(columns={'Abbreviation': 'State Abbreviations'}, inplace=True)

state_info = pd.merge(state_codes, state_abbreviations, left_on='State Names', right_on='State', how='outer')

state_info.drop(columns='State', inplace=True)
    
state_info.to_csv('data/county_info.csv', index=None)

state_info

## Create County Info Table

In [ ]:
# Create file with county info including names and codes

counties_json = pd.read_json('data/us_counties.json', encoding='latin1')

objects = counties_json.features.to_list()

state_codes = []
county_codes = []
county_names = []
county_census_area = []

for object in objects:
    p = object['properties']
    state_codes.append(p['STATE'])
    county_codes.append(p['COUNTY'])
    county_names.append(p['NAME'])
    county_census_area.append(p['CENSUSAREA'])

county_info = pd.DataFrame(
    {'State Codes': state_codes,
    'County Codes': county_codes,
    'County Names': county_names,
    'County Census Area': county_census_area
    })
    
county_info.to_csv('data/county_info.csv', index=None)

county_info

## UFO Data

In [ ]:
#UFO CSV Data (includes Canadian "States")

ufo_data = pd.read_csv('data/ufo_data.csv', low_memory=False)


In [ ]:
# Sighting frequencies for each state

sightings_by_state = ufo_data[['datetime', 'state']].groupby('state').count().reset_index()
sightings_by_state.columns = ['State', 'Sightings']
sightings_by_state

In [ ]:
ufo_data['duration (seconds)'] = pd.to_numeric(ufo_data['duration (seconds)'], errors='coerce')

print(ufo_data.info())

ufo_data[~ufo_data['duration (seconds)'].isnull()]

In [ ]:
# Average duration by State

duration_by_state = ufo_data[['duration (seconds)', 'state']].groupby('state').mean().reset_index()
duration_by_state.columns = ['State', 'Duration']
duration_by_state

### State Sightings and Duration

In [ ]:
state_sightings = pd.merge(sightings_by_state, duration_by_state, on='State')

state_sightings.to_csv('data/state_sightings.csv', index=None)

state_sightings

In [ ]:
ufo_data['shape'].value_counts()

In [ ]:
ufo_data['shape'] = ufo_data['shape'].apply(lambda x: 'other' if x in ('delta', 'crescent', 'round', 'pyramid', 'changed', 'hexagon', 'flare', 'dome') else x)
ufo_data['shape'].value_counts()

### Shapes Counts by State

In [ ]:
# Count of Shapes by State

shape_counts = pd.pivot_table(ufo_data, index='state', aggfunc='count', columns='shape')

# shape_counts = shape_counts.rename(columns = {''})

shape_counts_by_state = shape_counts.iloc[:, 0:21].reset_index()
shape_counts_by_state.columns = shape_counts_by_state.columns.droplevel()

cols = ['State', 'changing', 'chevron', 'cigar', 'circle', 'cone', 'cross',
       'cylinder', 'diamond', 'disk', 'egg', 'fireball', 'flash', 'formation',
       'light', 'other', 'oval', 'rectangle', 'sphere', 'teardrop', 'triangle',
       'unknown']

new_cols = []
for c in cols:
    new_cols.append(c.title())

shape_counts_by_state.columns = new_cols

shape_counts_by_state.to_csv('data/state_shape_counts.csv', index=None)

## Converting into Heatmap of States by Sighting with Python

In [ ]:
# Configure state maps

state_geo = json.load(open("data/us_states.json"))

print(len(state_geo['features']))
print(state_geo.keys())
print()

In [ ]:
# # Configure county maps

# county_geo = json.load(open("data/us_counties.json"))

# print(len(county_geo['features']))
# print(county_geo.keys())
# print()

In [ ]:
data_dict = sightings_by_state.set_index('State').to_dict()['Sightings']
rate_max = max(data_dict.values())
rate_min = min(data_dict.values())
nh_range = rate_max - rate_min

In [ ]:
#Scale the states values to lie between 0 and 1
min_nh = min(data_dict.values())
max_nh = max(data_dict.values())
nh_range = max_nh - min_nh

def calculate_color(state): #Convert the state sightings to a color
    normalized_nh = (state - min_nh) / nh_range # make the state sightings a number between 0 and 1
    inverse_nh = 1.0 - normalized_nh # invert state sightings so that a higher values renders a darker color
    mpl_color = plasma(inverse_nh) # transform the neighborhood value to a matplotlib color
    gmaps_color = to_hex(mpl_color, keep_alpha=False) # transform from a matplotlib color to a valid CSS color
    return gmaps_color

colors = []
for feature in state_geo['features']:
    geo_nh_name = feature['properties']['NAME']
    try:
        nh = data_dict[geo_nh_name]
        color = calculate_color(nh)
    except KeyError:
        # no value for that state: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [ ]:
fig = gmaps.figure()
nh_layer = gmaps.geojson_layer(
    state_geo,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig.add_layer(nh_layer)
fig